In [ ]:
import ldap
from ldap.controls import SimplePagedResultsControl
import pprint
import datetime as dt

In [ ]:
conn = ldap.initialize('ldap://ipaddress:port')
ou = "dc=test,dc=com"
conn.set_option(ldap.OPT_REFERRALS, 0)
conn.protocol_version = 3
conn.simple_bind_s('username', 'password')
PARAMS = ["distinguishedName",'sAMAccountName','givenName','sn','manager']
page_size = 1000
lc = SimplePagedResultsControl(True,size=page_size,cookie='')
known_ldap_resp_ctrls = {
SimplePagedResultsControl.controlType:SimplePagedResultsControl,
}

In [ ]:
msgid = conn.search_ext(ou, ldap.SCOPE_SUBTREE , "(&(objectClass=person)(employeeId=*))",PARAMS, serverctrls=[lc])
pages = 0
while True:
    pages += 1
    rtype, rdata, rmsgid, serverctrls = conn.result3(msgid,resp_ctrl_classes=known_ldap_resp_ctrls)
    user = {}
    for each in rdata:
        user = {}
        login_id, first_name, last_name, manager = ["","","",""]
        try:
            login_id = each[1]['sAMAccountName'][0]
        except:
            pass      
        try:
            first_name = each[1]['givenName'][0]
        except:
            pass
        try:
            last_name = each[1]['sn'][0]
        except:
            pass   
        try:
            manager = each[1]['manager'][0].split(',')[0].split('=')[1] 
        except:
            pass
        user['mmt_id'] = login_id
        user['username'] = login_id
        user['full_name'] = first_name + " " + last_name
        user['manager'] = manager
        user['datetime'] = dt.datetime.utcnow()
        user['application'] = 'AD'
        user['alternate_usernames'] = [ login_id , first_name + last_name[0:2] , first_name + last_name[0:1] , first_name[0:1] + last_name , first_name ]
        print user
        break
        
        #pprint.pprint(user)
    pctrls = [
      c
      for c in serverctrls
      if c.controlType == SimplePagedResultsControl.controlType
    ]
    if pctrls:
        if pctrls[0].cookie:
            lc.cookie = pctrls[0].cookie
            msgid = conn.search_ext('dc=mmt,dc=com', ldap.SCOPE_SUBTREE , "(&(objectClass=person)(employeeId=*))",PARAMS, serverctrls=[lc])
        else:
            break
    else:
        print "Warning: Server ignores RFC 2696 control."
        break